In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年5月28日

LULESHのプロファイルから予測をする

In [3]:
date :str = "2022年6月1日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

# 集合演算を用いて説明変数・目的変数が入力DFに存在していることを確認
if input_list_expVarNames == [] or (set(input_list_expVarNames).issubset(set(input_rawDF.columns.tolist())) == False):
    warnings.warn(f"説明変数の指定に誤りがあります")
    # return False
if input_list_resVarNames == [] or (set(input_rawDF.columns.tolist()) == False):
    warnings.warn(f"問題変数の指定に誤りがあります")
    # return False

function_names :list[str] = list(set(input_rawDF["functionName"].tolist()))
if len(function_names) == 0:
    warnings.warn("与えられたDFに関数名がありません")
    # return False

result_series_list :list[pd.Series] = []

for function_name in function_names:
    # 関数ごとの生データ
    input_rawDF_per_function = input_rawDF[input_rawDF["functionName"] == function_name]
    # モデルの構築
    models = Models(
        inputDF = input_rawDF_per_function,
        expVarColNames=input_list_expVarNames,
        resVarColNames=input_list_resVarNames,
        targetDF=None,
        modelNames=model_names,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値をfloatにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    # 関数ごとの結果に格納
    dict_for_series :dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF :pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(inputDF=resultDF, model_name_list=model_names, version=2)


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

In [4]:
resultDF

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,functionName,最低値
0,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::SetupCommBuffers(Int_t),0.000000e+00
1,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,void_CommSyncPosVel(Domain,9.075633e-15
2,3.085659e+02,0.000074,243.777234,1.808193e+02,6.014111e+01,308.565885,308.565885,7.382729e-05,0.000074,2.437772e+02,243.777234,346.822183,1.079287e-04,void_VerifyAndWriteFinalOutput(Real_t_Domain,7.382729e-05
3,0.000000e+00,0.000000,0.000000,1.782144e-08,1.642083e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,StrToInt,0.000000e+00
4,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,.TAU_application,0.000000e+00
5,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::CreateRegionIndexSets(Int_t_Int_t),0.000000e+00
6,1.500754e-13,172.922882,111.693928,8.499044e+01,7.624396e+01,172.922882,111.693928,7.287886e-14,111.693928,1.520599e-13,172.922882,113.765030,1.625247e-12,void_CommSend(Domain,7.287886e-14
7,1.510565e-13,164.783115,105.457903,7.622791e+01,7.563110e+01,164.783115,105.457903,7.687273e-15,105.457903,3.577109e-14,164.783115,107.747783,2.030330e-12,void_CommSBN(Domain,7.687273e-15
8,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_InitMeshDecomp(Int_t_Int_t_Int_t,0.000000e+00
9,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,void_CommMonoQ(Domain,9.075633e-15


In [5]:
resultDF.mean()

/tmp/ipykernel_13550/2794288614.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  resultDF.mean()


modelLin                               324.173667
modelIp                                411.497704
modelLog                               454.526334
modelProcessDividedByProblemSize        40.081234
modelProblemSizeDividedByProcess        35.645728
modelLinAndIp                          421.093841
modelLinAndLog                         455.735629
modelIpAndLin                          315.660772
modelIpAndLog                          446.710080
modelLogAndLin                         323.795580
modelLogAndIp                          419.026472
modelLinearSumOf2elementCombination    194.468150
modelLinearSumOfElementCombinations     41.843775
最低値                                      4.756974
dtype: float64

In [6]:
DF_funcName_lowestMAPE :pd.DataFrame = resultDF.filter(items=["functionName", "最低値"])
DF_funcName_lowestMAPE = DF_funcName_lowestMAPE.rename(columns={"functionName":"関数名", "最低値":"最低MAPE"})

print(DF_funcName_lowestMAPE.style.format(precision=2).to_latex(
    caption=f"ベンチマークプログラムLULESHの関数における最低MAPE",
    label=f"{date}lowestMAPEinLULESH"
))

\begin{table}
\caption{ベンチマークプログラムLULESHの関数における最低MAPE}
\label{2022年6月1日lowestMAPEinLULESH}
\begin{tabular}{llr}
 & 関数名 & 最低MAPE \\
0 & void_Domain::SetupCommBuffers(Int_t) & 0.00 \\
1 & void_CommSyncPosVel(Domain & 0.00 \\
2 & void_VerifyAndWriteFinalOutput(Real_t_Domain & 0.00 \\
3 & StrToInt & 0.00 \\
4 & .TAU_application & 0.00 \\
5 & void_Domain::CreateRegionIndexSets(Int_t_Int_t) & 0.00 \\
6 & void_CommSend(Domain & 0.00 \\
7 & void_CommSBN(Domain & 0.00 \\
8 & void_InitMeshDecomp(Int_t_Int_t_Int_t & 0.00 \\
9 & void_CommMonoQ(Domain & 0.00 \\
10 & MPI_Allreduce() & 0.00 \\
11 & void_Domain::~Domain() & 0.00 \\
12 & void_CalcKinematicsForElems(Domain & 0.00 \\
13 & MPI_Wait() & 3.87 \\
14 & MPI_Comm_rank() & 0.00 \\
15 & MPI_Waitall() & 0.00 \\
16 & void_Domain::SetupElementConnectivities(Int_t) & 0.00 \\
17 & MPI_Comm_size() & 0.00 \\
18 & void_Domain::SetupSymmetryPlanes(Int_t) & 0.00 \\
19 & void_Domain::Domain(Int_t_Index_t_Index_t_Index_t_Index_t_Int_t_Int_t_Int_t_Int_t) & 0.

# 2022年5月25日～

重み付き平均の実装

todos
* ✅✅"modelLin":ModelLinForMultipleRegression
* ✅✅"modelIp":ModelIpForMultipleRegression
* ✅✅"modelLog":ModelLogForMultipleRegression
* ✅✅"modelLinAndIp":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLinAndLog":ModelMultipleEquationForMultipleRegression
* ✅✅"modelIpAndLin":ModelMultipleEquationForMultipleRegression
* ✅✅"modelIpAndLog":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLogAndLin":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLogAndIp":ModelMultipleEquationForMultipleRegression
* ✅✅"modelProcessDividedByProblemSize":Model_ProcessesDevidedByProblemSize_ForMultipleRegression
* ✅✅"modelProblemSizeDividedByProcess":Model_ProblemSizeDevidedByProcesses_ForMultipleRegression
* ✅✅"modelLinearSumOf2elementCombination":Model_LinearSumOf2elementCombination_ForMultipleRegression
* ✅✅"modelLinearSumOfElementCombinations":Model_LinearSumOfElementCombinations_ForMultipleRegression

* `class Models`での対応
    * ✅models.calcWeightedMAPE()
    * ✅models.returnCalculatedMAPE()

In [7]:
date :str = "2022年6月1日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

# 集合演算を用いて説明変数・目的変数が入力DFに存在していることを確認
if input_list_expVarNames == [] or (set(input_list_expVarNames).issubset(set(input_rawDF.columns.tolist())) == False):
    warnings.warn(f"説明変数の指定に誤りがあります")
    # return False
if input_list_resVarNames == [] or (set(input_rawDF.columns.tolist()) == False):
    warnings.warn(f"問題変数の指定に誤りがあります")
    # return False

function_names :list[str] = list(set(input_rawDF["functionName"].tolist()))
if len(function_names) == 0:
    warnings.warn("与えられたDFに関数名がありません")
    # return False

result_series_list :list[pd.Series] = []

for function_name in function_names:
    # 関数ごとの生データ
    input_rawDF_per_function = input_rawDF[input_rawDF["functionName"] == function_name]
    # モデルの構築
    models = Models(
        inputDF = input_rawDF_per_function,
        expVarColNames=input_list_expVarNames,
        resVarColNames=input_list_resVarNames,
        targetDF=None,
        modelNames=model_names,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcWeightedMAPE()

    dictCalcedWeightedMAPE = models.returnCalculatedWeightedMAPE()

    # 算出されたMAPEの数値をfloatにする
    for key in dictCalcedMAPE.keys():
        dictCalcedWeightedMAPE[key] = float(dictCalcedWeightedMAPE[key])

    # 関数ごとの結果に格納
    dict_for_series :dict = copy.deepcopy(dictCalcedWeightedMAPE)
    dict_for_series["functionName"] = function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF :pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(inputDF=resultDF, model_name_list=model_names, version=2)


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

In [8]:

resultDF

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,functionName,最低値
0,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,void_Domain::SetupCommBuffers(Int_t),0.000000e+00
1,4.972507e-16,6.351914e-01,0.384580,7.691018e-01,7.208057e-01,0.401861,6.762962e-15,0.635191,0.384580,2.123102e-17,3.845805e-01,1.107706e-16,0.635191,void_CommSyncPosVel(Domain,2.123102e-17
2,9.453149e-01,3.144336e-07,0.615327,9.827857e-01,5.219435e-01,0.966441,3.395252e-07,0.945315,0.945315,3.144336e-07,3.144336e-07,6.153269e-01,0.615327,void_VerifyAndWriteFinalOutput(Real_t_Domain,3.144336e-07
3,0.000000e+00,0.000000e+00,0.000000,1.782144e-10,1.642083e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,StrToInt,0.000000e+00
4,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,.TAU_application,0.000000e+00
5,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,void_Domain::CreateRegionIndexSets(Int_t_Int_t),0.000000e+00
6,5.180623e-16,6.326808e-01,0.383060,8.314608e-01,7.179552e-01,0.400273,5.305109e-15,0.632681,0.383060,2.085734e-16,3.830604e-01,4.804489e-16,0.632681,void_CommSend(Domain,2.085734e-16
7,4.906958e-16,6.277186e-01,0.380056,7.600546e-01,7.123192e-01,0.397134,6.682236e-15,0.627719,0.380056,1.633718e-17,3.800560e-01,6.568042e-17,0.627719,void_CommSBN(Domain,1.633718e-17
8,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,void_InitMeshDecomp(Int_t_Int_t_Int_t,0.000000e+00
9,4.972507e-16,6.351914e-01,0.384580,7.691018e-01,7.208057e-01,0.401861,6.762962e-15,0.635191,0.384580,2.123102e-17,3.845805e-01,1.107706e-16,0.635191,void_CommMonoQ(Domain,2.123102e-17


In [9]:
resultDF.mean()

/tmp/ipykernel_13550/2794288614.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  resultDF.mean()


modelLin                               0.087942
modelIp                                0.284129
modelLog                               0.209925
modelProcessDividedByProblemSize       0.370852
modelProblemSizeDividedByProcess       0.337622
modelLinearSumOf2elementCombination    0.199110
modelLinearSumOfElementCombinations    0.018873
modelLinAndIp                          0.313909
modelLinAndLog                         0.220870
modelIpAndLin                          0.057056
modelIpAndLog                          0.189664
modelLogAndLin                         0.075889
modelLogAndIp                          0.302322
最低値                                    0.018873
dtype: float64

In [10]:
DF_funcName_lowestMAPE :pd.DataFrame = resultDF.filter(items=["functionName", "最低値"])
DF_funcName_lowestMAPE = DF_funcName_lowestMAPE.rename(columns={"functionName":"関数名", "最低値":"最低MAPE"})

print(DF_funcName_lowestMAPE.style.format(precision=2).to_latex(
    caption=f"ベンチマークプログラムLULESHの関数における最低MAPE",
    label=f"{date}lowestMAPEinLULESH"
))

\begin{table}
\caption{ベンチマークプログラムLULESHの関数における最低MAPE}
\label{2022年6月1日lowestMAPEinLULESH}
\begin{tabular}{llr}
 & 関数名 & 最低MAPE \\
0 & void_Domain::SetupCommBuffers(Int_t) & 0.00 \\
1 & void_CommSyncPosVel(Domain & 0.00 \\
2 & void_VerifyAndWriteFinalOutput(Real_t_Domain & 0.00 \\
3 & StrToInt & 0.00 \\
4 & .TAU_application & 0.00 \\
5 & void_Domain::CreateRegionIndexSets(Int_t_Int_t) & 0.00 \\
6 & void_CommSend(Domain & 0.00 \\
7 & void_CommSBN(Domain & 0.00 \\
8 & void_InitMeshDecomp(Int_t_Int_t_Int_t & 0.00 \\
9 & void_CommMonoQ(Domain & 0.00 \\
10 & MPI_Allreduce() & 0.00 \\
11 & void_Domain::~Domain() & 0.00 \\
12 & void_CalcKinematicsForElems(Domain & 0.00 \\
13 & MPI_Wait() & 0.04 \\
14 & MPI_Comm_rank() & 0.00 \\
15 & MPI_Waitall() & 0.00 \\
16 & void_Domain::SetupElementConnectivities(Int_t) & 0.00 \\
17 & MPI_Comm_size() & 0.00 \\
18 & void_Domain::SetupSymmetryPlanes(Int_t) & 0.00 \\
19 & void_Domain::Domain(Int_t_Index_t_Index_t_Index_t_Index_t_Int_t_Int_t_Int_t_Int_t) & 0.

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 
